In [1]:
import pickle
import re
from nltk.tokenize import TweetTokenizer
from nltk import ngrams
import numpy as np
from sklearn import svm
from sklearn import metrics


In [2]:

train_set = np.asarray(pickle.load(open("train_set.p", "rb")))
test_set = np.asarray(pickle.load(open("test_set.p", "rb")))


In [3]:
y_train = train_set[:, 1]
train = train_set[:, 0]

y_test = test_set[:, 1]
test = test_set[:, 0]


In [4]:
#print(train_set[30:40])

In [5]:
def get_num_of_hastags(tweet):
    return tweet.count("#")


In [6]:
def get_num_of_mult_punctuation(tweet):
    return len(re.findall(r"[?!]{2,}", tweet))


In [7]:
def get_ngrams(tweet, n):
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(tweet)
    
    l = []
    for ngram in ngrams(tokens, n):
        l.append(' '.join(str(i) for i in ngram))

    return l


In [8]:
def extract_features(data_set):
    X = np.zeros((len(data_set), 2))    

    for i, tweet in enumerate(data_set): 
        num_hastags = get_num_of_hastags(tweet)
        num_mult_punct = get_num_of_mult_punctuation(tweet)

        X[i] = np.asarray([num_hastags, num_mult_punct])
    
    return X

X_train = extract_features(train)
X_test = extract_features(test)


In [10]:
lin_svm = svm.LinearSVC()
lin_svm.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [11]:
prediction = lin_svm.predict(X_test)

In [35]:
metrics.accuracy_score(y_test, prediction)

0.20499999999999999